In [1]:
import numpy as np
import pandas as pd
import sqlalchemy

from database_credentials import get_database_url
from datetime import datetime, timedelta

In [2]:
def drop_features(data):
    """Drop features not related to turnip prices with
    the exception of In-Game Name and Island
    
    Args: 
        data (dataframe): 
            original data from Maddox Knight's Turnip Mafia
    
    Returns:
        data (dataframe): 
            contains In-Game Name, Island along with the 
            buying and selling prices of turnips
    """
    
    column_names = [
        'In-Game Name', 'Island', 'Buy Price', 'Mon AM', 
        'Mon PM', 'Tue AM', 'Tue PM', 'Wed AM', 'Wed PM', 
        'Thu AM', 'Thu PM', 'Fri AM','Fri PM','Sat AM','Sat PM'
    ]
    data = data[column_names]
    return data

def mask_invalid_names(data):
    """Mask rows in which In-Game Name or Island are invalid
    
    Args: 
        data (dataframe): 
            contains data associated with invalid In-Game Names 
            or Island
    
    Returns:
        data (dataframe): 
            contains data associated with both a valid In-Game Name 
            and Island
    """
    
    invalid_ign_mask = data['In-Game Name'].notna()
    invalid_island_mask = data['Island'].notna()
    valid_data = data[invalid_ign_mask & invalid_island_mask].copy()
    valid_data.reset_index(drop=True, inplace=True)
    return valid_data

def convert_entry_to_float(entry):
    """Convert a price entry to float while replacing invalid
    prices with NaN
    
    Args:
        entry (str/float):
            entry to be converted
    
    Returns:
        convert_entry (float): 
            entry as a float or np.nan 
    """
    
    try: 
        converted_entry = float(entry)
    except:
        converted_entry = np.nan
    return converted_entry

def preprocess(data):
    """Preprocess data by dropping features, masking invalid 
    In-game Name and Islands along with formatting turnip prices
    
    Args: 
        data (dataframe): 
            original data from Maddox Knight's Turnip Mafia
        
    Returns:
        preprocessed_data (dataframe): 
            contains In-Game Name, Island along with the 
            buying and selling prices of turnips
    """
    
    preprocessed_data = drop_features(data)
    preprocessed_data = mask_invalid_names(preprocessed_data)
    # week 4 Lala Hyazinth duplicated, drop duplicates
    preprocessed_data.drop_duplicates(
        subset=['In-Game Name', 'Island'], 
        keep='last', 
        inplace=True
    )
    
    price_columns = preprocessed_data.columns[2:]
    preprocessed_data[price_columns] = (
        preprocessed_data[price_columns].applymap(convert_entry_to_float)
    )
        
    return preprocessed_data

def get_buy_date():
    """Get the turnip buy date

    Args:
        None
    
    Returns:
        buy_dt (datetime): turnip buy date in %Y-%m-%d format
    """
    
    date_is_invalid = True
    while date_is_invalid:
        try:
            buy_date = input("Buy Date of Turnips (yyyy-mm-dd): ")
            buy_dt = datetime.strptime(buy_date, '%Y-%m-%d')
            date_is_invalid = False
        except:
            print("Invalid date format.")

    return buy_dt

def get_price_data(data, price_column, date):
    """Get the prices from a column and format it into a 
    dataframe columns containing In-Game Name, Island, 
    price and date 
    
    Args: 
        data (dataframe): preprocessed data
        price_column (str): column containing prices
        date (datetime): date prices were observed
    
    Returns:
        prices (dataframe): 
            contains In-Game Name, Island, turnip prices and date 
    """
    
    column_names = data.columns[0:2].to_list()
    column_names.append(price_column)
    price_data = pd.DataFrame(data[column_names])
    price_data.dropna(inplace=True)
    price_data['datetime_observed'] = date
    
    new_column_names = {
        'In-Game Name': 'user_id', 
        'Island': 'island_id',
        price_column: 'price'
    }
    
    price_data.rename(columns=new_column_names, inplace=True)
    return price_data

def get_buy_sell_price_data(data, buy_date):
    """Get the buying and selling prices of turnips from data
    
    Args:
        data (dataframe): preprocessed data
        buy_date (datetime): buy date of turnips in yyyy-mm-dd format
    
    Returns:
        buy_prices (dataframe): buying prices of turnips
        sell_prices (dataframe): selling prices of turnips
    """
    
    buy_data = get_price_data(data, 'Buy Price', buy_date)
    
    # slice Mon AM, Mon PM, ... Sat AM, Sat PM
    sell_columns = data.columns[3:]
    first_sell_dt = buy_date + timedelta(days=1)
    
    # list comp yyyy-mm-dd 00:00:00 for AM and 
    # yyyy-mm-dd 12:00:00 for PM times
    sell_times = [
        first_sell_dt + timedelta(hours=12*elapsed_interval) 
        for elapsed_interval in range(sell_columns.shape[0])
    ]
    
    # get data for each column and datetime pair
    sell_data = [
        get_price_data(data, column, date) 
        for column, date in zip(sell_columns, sell_times)
    ]
    sell_data = pd.concat(sell_data)
    
    return buy_data, sell_data

In [22]:
# data = pd.read_csv('data/week1.csv', skiprows=[0]) # 2020-04-12
# data = pd.read_csv('data/week2.csv', skiprows=[0, 1]) # 2020-04-19
# data = pd.read_csv('data/week3.csv', skiprows=[0, 1]) # 2020-04-26
data = pd.read_csv('data/week4.csv', skiprows=[0, 1]) # 2020-05-03

The goal of this exploration is to model the buying and selling prices of turnips in Animal Crossing: New Horizons. 

Preprocessing involves removing columns that do not relate to turnip prices and rows with invalid In-Game Name or Island entries.Invalid turnip prices will be replaced with NaNs while turnip prices in string format will be  cast into floats using try and except in conjunction with applymap. Note that this would be inefficient for larger datasets and that a more efficient approach would be to only target specific columns as opposed to the entire dataframe.

In [23]:
preprocessed_data = preprocess(data)

In [24]:
display(preprocessed_data.head(3))

,In-Game Name,Island,Buy Price,Mon AM,Mon PM,Tue AM,Tue PM,Wed AM,Wed PM,Thu AM,Thu PM,Fri AM,Fri PM,Sat AM,Sat PM
0,Maddox,Knight,102.0,114.0,130.0,167.0,180.0,155.0,62.0,58.0,NaN,NaN,NaN,NaN,NaN
1,Aika,Erijima,100.0,90.0,86.0,83.0,78.0,73.0,69.0,114.0,149.0,509.0,149.0,136.0,87.0
2,Riadh,Shmeow,95.0,80.0,108.0,181.0,517.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
buy_date = get_buy_date()
buy_data, sell_data = get_buy_sell_price_data(preprocessed_data, buy_date)

Buy Date of Turnips (yyyy-mm-dd): 2020-05-03


In [26]:
display(buy_data.head(3))
display(sell_data)

,user_id,island_id,price,datetime_observed
0,Maddox,Knight,102.0,2020-05-03
1,Aika,Erijima,100.0,2020-05-03
2,Riadh,Shmeow,95.0,2020-05-03


,user_id,island_id,price,datetime_observed
0,Maddox,Knight,114.0,2020-05-04 00:00:00
1,Aika,Erijima,90.0,2020-05-04 00:00:00
2,Riadh,Shmeow,80.0,2020-05-04 00:00:00
4,Dave,Alethela,115.0,2020-05-04 00:00:00
5,Vinz,Long Island,73.0,2020-05-04 00:00:00
...,...,...,...,...
45,Donovan,Macaroni,108.0,2020-05-09 12:00:00
48,Cee,Pupeen,75.0,2020-05-09 12:00:00
51,su tart,inc foot,68.0,2020-05-09 12:00:00
65,Anna,Sunsetleaf,50.0,2020-05-09 12:00:00


In [8]:
database_url = get_database_url()
engine = sqlalchemy.create_engine(database_url)
connection = engine.connect()

In [27]:
buy_data.to_sql('buy_data', connection, if_exists='append', index=False)
sell_data.to_sql('sell_data', connection, if_exists='append', index=False)

In [28]:
connection.close()
engine.dispose()